In [1]:
# Import needed libraries
import pandas as pd
import numpy as np
import arcpy
import os

#### I. Read in CSV Files

In [2]:
# Read in the 'presidential_election_results_2008_through_2016.csv' file
url = 'https://raw.githubusercontent.com/MXMcKnight/data/master/presidential_election_results_2008_through_2016.csv'
df_presidential_election_results = pd.read_csv(url, dtype = {'geoid':np.str})

# Use 'geoid' as the unique identifier
df_presidential_election_results = df_presidential_election_results.set_index('geoid')
df_presidential_election_results.head(3)

,county,total_2008,dem_2008,gop_2008,oth_2008,total_2012,dem_2012,gop_2012,oth_2012,total_2016,dem_2016,gop_2016,oth_2016
geoid,,,,,,,,,,,,,
26041,Delta County,19064,9974,8763,327,18043,8330,9533,180,18467,6431,11112,924
48295,Lipscomb County,1256,155,1093,8,1168,119,1044,5,1322,135,1159,28
01127,Walker County,28652,7420,20722,510,28497,6551,21633,313,29243,4486,24208,549


In [3]:
# Read in the 'flu_vaccination_percentages.csv' file
url = 'https://raw.githubusercontent.com/MXMcKnight/data/master/flu_vaccination_percentages_2016.csv'
df_flu_vax_pct = pd.read_csv(url, dtype = {'fips':np.str})

# Use 'fips' as the unique identifier
df_flu_vax_pct = df_flu_vax_pct.set_index('fips')
df_flu_vax_pct.head(3)

,state,county,flu_vax_pct,flu_vax_pct_black,flu_vax_pct_hispanic,flu_vax_pct_white
fips,,,,,,
01001,Alabama,Autauga,41.0,32.0,36.0,42.0
01003,Alabama,Baldwin,45.0,33.0,35.0,46.0
01005,Alabama,Barbour,37.0,33.0,NaN,38.0


In [4]:
# Read in the 'total_population_2016.csv' file
url = 'https://raw.githubusercontent.com/MXMcKnight/data/master/total_population_2016.csv'
df_total_population = pd.read_csv(url, dtype = {'geoid':np.str}, encoding = 'latin-1')

# Use 'geoid' as the unique identifier
df_total_population = df_total_population.set_index('geoid')
df_total_population.head(3)

,county,state,total_population
geoid,,,
01001,Autauga County,Alabama,55049
01003,Baldwin County,Alabama,199510
01005,Barbour County,Alabama,26614


#### II. Create Data Frame

In [5]:
# Prepare a data frame with fields needed for analysis 
df = pd.DataFrame()
df['total_pop'] = df_total_population['total_population']
df['trump_pct_2016'] = (df_presidential_election_results['gop_2016'] / 
                        df_presidential_election_results['total_2016']) * 100
df['clinton_pct_2016'] = (df_presidential_election_results['dem_2016'] / 
                          df_presidential_election_results['total_2016']) * 100
df['flu_vax_pct'] = df_flu_vax_pct['flu_vax_pct']
df['flu_vax_pct_black'] = df_flu_vax_pct['flu_vax_pct_black']
df['flu_vax_pct_hispanic'] = df_flu_vax_pct['flu_vax_pct_hispanic']
df['flu_vax_pct_white'] = df_flu_vax_pct['flu_vax_pct_white']
df.head(3)

,total_pop,trump_pct_2016,clinton_pct_2016,flu_vax_pct,flu_vax_pct_black,flu_vax_pct_hispanic,flu_vax_pct_white
geoid,,,,,,,
01001,55049,73.435789,23.956855,41.0,32.0,36.0,42.0
01003,199510,77.351472,19.565310,45.0,33.0,35.0,46.0
01005,26614,52.271415,46.660250,37.0,33.0,NaN,38.0


#### III. Identify Columns with Missing or Peculiar Values

In [6]:
# Determine which columns have missing values
df.isna().any()

total_pop               False
trump_pct_2016           True
clinton_pct_2016         True
flu_vax_pct              True
flu_vax_pct_black        True
flu_vax_pct_hispanic     True
flu_vax_pct_white        True
dtype: bool

In [7]:
# Query 'trump_pct_2016' column for peculiar values
print('Negative Numbers Out of Bounds:', np.any(df['trump_pct_2016'] < 0))
print('Positive Numbers Out of Bounds:', np.any(df['trump_pct_2016'] > 100))
print('Null Values:', np.any(df['trump_pct_2016'].isnull()))
print('NaNs:', np.any(np.isnan(df['trump_pct_2016'])))

Negative Numbers Out of Bounds: False
Positive Numbers Out of Bounds: False
Null Values: True
NaNs: True


In [8]:
# Query 'clinton_pct_2016' column for peculiar values
print('Negative Numbers Out of Bounds:', np.any(df['clinton_pct_2016'] < 0))
print('Positive Numbers Out of Bounds:', np.any(df['clinton_pct_2016'] > 100))
print('Null Values:', np.any(df['clinton_pct_2016'].isnull()))
print('NaNs:', np.any(np.isnan(df['clinton_pct_2016'])))

Negative Numbers Out of Bounds: False
Positive Numbers Out of Bounds: False
Null Values: True
NaNs: True


In [9]:
# Query 'flu_vax_pct' column for peculiar values
print('Negative Numbers Out of Bounds:', np.any(df['flu_vax_pct'] < 0))
print('Positive Numbers Out of Bounds:', np.any(df['flu_vax_pct'] > 100))
print('Null Values:', np.any(df['flu_vax_pct'].isnull()))
print('NaNs:', np.any(np.isnan(df['flu_vax_pct'])))

Negative Numbers Out of Bounds: False
Positive Numbers Out of Bounds: False
Null Values: True
NaNs: True


In [10]:
# Query 'flu_vax_pct_black' column for peculiar values
print('Negative Numbers Out of Bounds:', np.any(df['flu_vax_pct_black'] < 0))
print('Positive Numbers Out of Bounds:', np.any(df['flu_vax_pct_black'] > 100))
print('Null Values:', np.any(df['flu_vax_pct_black'].isnull()))
print('NaNs:', np.any(np.isnan(df['flu_vax_pct_black'])))

Negative Numbers Out of Bounds: False
Positive Numbers Out of Bounds: False
Null Values: True
NaNs: True


In [11]:
# Query 'flu_vax_pct_hispanic' column for peculiar values
print('Negative Numbers Out of Bounds:', np.any(df['flu_vax_pct_hispanic'] < 0))
print('Positive Numbers Out of Bounds:', np.any(df['flu_vax_pct_hispanic'] > 100))
print('Null Values:', np.any(df['flu_vax_pct_hispanic'].isnull()))
print('NaNs:', np.any(np.isnan(df['flu_vax_pct_hispanic'])))

Negative Numbers Out of Bounds: False
Positive Numbers Out of Bounds: False
Null Values: True
NaNs: True


In [12]:
# Query 'flu_vax_pct_white' column for peculiar values
print('Negative Numbers Out of Bounds:', np.any(df['flu_vax_pct_white'] < 0))
print('Positive Numbers Out of Bounds:', np.any(df['flu_vax_pct_white'] > 100))
print('Null Values:', np.any(df['flu_vax_pct_white'].isnull()))
print('NaNs:', np.any(np.isnan(df['flu_vax_pct_white'])))

Negative Numbers Out of Bounds: False
Positive Numbers Out of Bounds: False
Null Values: True
NaNs: True


In [14]:
df = df.fillna(value = -1)

In [17]:
# Export data frame with missing values
flu_vaccination_percentages_nan_values = df.to_csv(r'C:\Advanced Analysis in GIS\Flu Vaccinations\Data' + 
                                                   r'\flu_vaccination_percentages_nan_values.csv')